In [9]:
import pandas as pd
from difflib import SequenceMatcher

In [10]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [11]:
left_data_path = '../data/left_dataset.csv'
right_data_path = '../data/right_dataset.csv'
left_data = pd.read_csv(left_data_path)
right_data = pd.read_csv(right_data_path)

In [12]:
left_data.dropna(inplace=True)
right_data.dropna(inplace=True)
print("Missing values in left_df:")
print(left_data.isnull().sum())

print("Missing values in right_data:")
print(right_data.isnull().sum())

Missing values in left_df:
entity_id      0
name           0
address        0
city           0
state          0
postal_code    0
categories     0
dtype: int64
Missing values in right_data:
business_id    0
name           0
address        0
city           0
state          0
zip_code       0
size           0
dtype: int64


In [13]:

#change all name and address to string and be lowercase
left_data['name'] = left_data['name'].astype(str).str.lower()
left_data['address'] = left_data['address'].astype(str).str.lower()
right_data['name'] = right_data['name'].astype(str).str.lower()
right_data['address'] = right_data['address'].astype(str).str.lower()
#remove punctuation
left_data['postal_code'] = left_data['postal_code'].astype(str).apply(lambda x: x.split('.')[0])
right_data['zip_code'] = right_data['zip_code'].astype(str).apply(lambda x: x.split('-')[0])


In [14]:
left_data['block_key'] = left_data['name'].str[:2] + left_data['address'].str[0] + left_data['state'] + left_data['postal_code'].str[:3]
right_data['block_key'] = right_data['name'].str[:2] + right_data['address'].str[0] + right_data['state'] + right_data['zip_code'].str[:3]

In [15]:
merged = pd.merge(left_data, right_data, on='block_key', how='inner', suffixes=('_left', '_right'))

In [16]:
merged.head()

,entity_id,name_left,address_left,city_left,state_left,postal_code,categories,block_key,business_id,name_right,address_right,city_right,state_right,zip_code,size
0,1,the ups store,87 grasso plaza shopping center,Affton,MO,63123,"Shipping Centers, Local Services, Notaries, Ma...",th8MO631,37027,the art of entertaining inc,8796 big bend blvd,SAINT LOUIS,MO,63119,25.0
1,1,the ups store,87 grasso plaza shopping center,Affton,MO,63123,"Shipping Centers, Local Services, Notaries, Ma...",th8MO631,37156,"the kohn partnership, llp",8251 maryland ave suite 108,SAINT LOUIS,MO,63105,9.0
2,1,the ups store,87 grasso plaza shopping center,Affton,MO,63123,"Shipping Centers, Local Services, Notaries, Ma...",th8MO631,37159,the schechter law firm pc,8000 maryland ave ste 950,Saint Louis,MO,63105,9.0
3,1,the ups store,87 grasso plaza shopping center,Affton,MO,63123,"Shipping Centers, Local Services, Notaries, Ma...",th8MO631,37162,"the schechter law firm, p.c.",8000 maryland suite 950,ST LOUIS,MO,63105,9.0
4,1,the ups store,87 grasso plaza shopping center,Affton,MO,63123,"Shipping Centers, Local Services, Notaries, Ma...",th8MO631,37221,"thomas l. dowell & associates, inc.",8460 watson rd suite 141,SAINT LOUIS,MO,63119,7.0


In [17]:
merged['name_similarity'] = merged.apply(lambda x: similar(x['name_left'], x['name_right']), axis=1)
merged['address_similarity'] = merged.apply(lambda x: similar(x['address_left'], x['address_right']), axis=1)

In [18]:
confidence = 0.5 * merged['name_similarity'] + 0.5 * merged['address_similarity']
merged['confidence'] = confidence
filtered_merged = merged[merged['confidence'] > 0.8]
print(filtered_merged[['entity_id', 'business_id', 'confidence']])


         entity_id  business_id  confidence
645          82635        50523    0.941860
646          82635        50540    0.931818
648          89051        37027    0.960000
799          24054        78876    0.916667
895          14584        39665    0.867347
...            ...          ...         ...
1133852      93910        18515    0.833333
1133858      94035        72787    0.944444
1133862      94065        81979    0.865385
1133865      94131        27269    0.925000
1133896      94546        83310    0.937500

[7534 rows x 3 columns]
